In [47]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
client.create_experiment(name="experiment_from_the_client")

'3'

In [8]:
# WE CAN ALSO QUERY THE LIST OF MLRUNS IN EXPERIMENT #1 WHICH ARE ONLY ACTIVE USING
# ASCENDING RMSE VALUE
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [23]:
for run in runs:
    print(f"Run Id: {run.info.run_id}, RMSE: {run.data.metrics['rmse']:.4f}")

Run Id: 7097015688ac41fbb1b910d577a4a461, RMSE: 12.2126
Run Id: d22fbe3d0aa044f1bc509432e24857a2, RMSE: 12.2126


KeyError: 'rmse'

In [32]:
model_name = "NYC-TAXI-REGRESSOR"
latest_versions = client.get_latest_versions(name=model_name)

In [33]:
latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1685116890797, current_stage='Staging', description='Worst Model', last_updated_timestamp=1685127425428, name='NYC-TAXI-REGRESSOR', run_id='4b986ce48442452e8093c5e3d7d5a170', run_link='', source='mlflow-artifacts:/2/4b986ce48442452e8093c5e3d7d5a170/artifacts/models_mlflow', status='READY', status_message='', tags={}, user_id='', version='2'>]

In [35]:
for version in latest_versions:
    print(f"version: {version.version} stage: {version.current_stage}")

version: 2 stage: Staging


MODULAR IMPLEMENTATION OF NYC-TAXI-DURATION PREDICTION

In [36]:
import mlflow

In [38]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = 'trip_distance'
    train_dicts = df[categorical + numerical].to_dict(orient='reversed')
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [39]:
df = read_dataframe('./data/green_tripdata_2022-03.parquet')

In [48]:
run_id = 'e4f61f89f2c74837ab1b69bcea486e5c'
client.download_artifacts(run_id=run_id, path='preprocessor')

/tmp/ipykernel_56615/327658439.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor')


MlflowException: The configured tracking uri scheme: 'file' is invalid for use with the proxy mlflow-artifact scheme. The allowed tracking schemes are: {'http', 'https'}